In [16]:
from PIL import Image
import numpy as np
import os
import cv2
import glob,random
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten ,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras

In [17]:
data_dir  = "F:/WorkSpace/Sign_Language_Testing/Words_testing/st_end/data/bnw"

In [18]:
classes   =  os.listdir(data_dir)
print(classes)

['brother', 'hello', 'i', 'learn', 'learner_1', 'learner_2', 'name', 'no']


In [19]:
count=0
data=[]
labels=[]

In [20]:
image_files = [f for f in glob.glob(data_dir + "/**/*", recursive=True) if not os.path.isdir(f)]
random.shuffle(image_files)

In [21]:
for img in image_files:
    count+=1
    image = cv2.imread(img)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = img_to_array(image)
    data.append(image)
    labels.append(img.split(os.path.sep)[-2]) 

In [22]:
print(len(labels))

['brother', 'no', 'i', 'learner_2', 'learner_1', 'name', 'no', 'learn', 'name', 'no', 'name', 'name', 'learner_2', 'no', 'name', 'learner_2', 'learn', 'learn', 'learner_1', 'learner_2', 'name', 'no', 'name', 'learner_2', 'no', 'learn', 'no', 'no', 'brother', 'brother', 'name', 'name', 'learner_1', 'i', 'learner_1', 'learner_1', 'learner_2', 'name', 'i', 'name', 'no', 'learner_2', 'name', 'no', 'name', 'learn', 'name', 'i', 'brother', 'no', 'learn', 'learn', 'learn', 'no', 'i', 'name', 'learner_2', 'learner_1', 'learner_1', 'learn', 'no', 'learner_1', 'learner_2', 'name', 'no', 'name', 'learner_1', 'learner_2', 'learner_1', 'name', 'name', 'learner_1', 'no', 'learner_1', 'learn', 'learn', 'name', 'learn', 'learn', 'name', 'learn', 'learner_2', 'learner_1', 'learner_2', 'learn', 'learner_2', 'i', 'hello', 'name', 'learner_2', 'i', 'name', 'learner_2', 'hello', 'no', 'learn', 'i', 'learner_2', 'i', 'no', 'learner_2', 'i', 'brother', 'name', 'no', 'learner_1', 'name', 'learner_2', 'name', 

In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
data = np.array(data, dtype="float")/255.

data=data.reshape(count,240,240,1)
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 8)
print(len(labels))

[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [25]:
print(data.shape)

(1516, 240, 240, 1)


In [26]:
print(len(data),len(labels))

1516 1516


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.10, random_state=40)

In [28]:
aug = ImageDataGenerator(
    width_shift_range=(0.2),
    height_shift_range=(0.2),
    rotation_range=(15),
    horizontal_flip=False,
    zoom_range=(0.9,1.3),
    fill_mode="nearest")

In [29]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D 
from keras.layers import Dense, Flatten ,GlobalMaxPool2D

In [38]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(240, 240, 1),padding="same")) 
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3 ,3), activation='relu') )
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3 ,3), activation='relu') )
model.add(MaxPooling2D((2, 2)))


model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

In [39]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
model.fit(x=aug.flow(X_train, y_train,batch_size=32),batch_size=32, epochs=20, validation_data=aug.flow(X_test, y_test))

Epoch 1/20
43/43 [==============================] - 5s 127ms/step - loss: 0.1586 - accuracy: 0.9575 - val_loss: 0.1138 - val_accuracy: 0.9737
Epoch 2/20
43/43 [==============================] - 6s 128ms/step - loss: 0.1350 - accuracy: 0.9567 - val_loss: 0.1132 - val_accuracy: 0.9474
Epoch 3/20
43/43 [==============================] - 6s 139ms/step - loss: 0.1171 - accuracy: 0.9648 - val_loss: 0.1067 - val_accuracy: 0.9671
Epoch 4/20
43/43 [==============================] - 6s 133ms/step - loss: 0.1572 - accuracy: 0.9545 - val_loss: 0.1572 - val_accuracy: 0.9408
Epoch 5/20
43/43 [==============================] - 6s 133ms/step - loss: 0.1524 - accuracy: 0.9582 - val_loss: 0.0725 - val_accuracy: 0.9868
Epoch 6/20
43/43 [==============================] - 6s 133ms/step - loss: 0.1076 - accuracy: 0.9677 - val_loss: 0.1367 - val_accuracy: 0.9539
Epoch 7/20
43/43 [==============================] - 6s 137ms/step - loss: 0.1092 - accuracy: 0.9677 - val_loss: 0.0519 - val_accuracy: 0.9803
Epoch 

In [48]:
save_dir = "F:/WorkSpace/Sign_Language_Testing/Words_testing/st_end/models/edge/sobel_bnw_3_II"
model.save(save_dir)

INFO:tensorflow:Assets written to: F:/WorkSpace/Sign_Language_Testing/Words_testing/st_end/models/edge/sobel_bnw_3_II\assets
